In [1]:
import numpy as np
from itertools import combinations
import random
import time

In [2]:
def is_feasible(solution, A, b, eps):
    b_est = A @ solution
    sse = np.sum((b - b_est)**2)
    return sse < eps

In [3]:
class Individual:
    def __init__(self, A, b, eps):            
        n = A.shape[1]
        self.cols = random.sample(range(n), random.randint(1, n))
        self.cols.sort()
        self.update_x(A, b)
        self.fitness = self.calc_fitness(A, b, eps)
    
    def update_x(self, A, b):
        n = A.shape[1]
        A_sub = A[:,self.cols]
        try:
            sol_sub = np.linalg.lstsq(A_sub, b, rcond=None)[0]
            self.x = np.zeros(n)
            self.x[self.cols] = sol_sub
        except np.linalg.LinAlgError:
            self.cols = random.sample(range(n), random.randint(1, n))
            self.cols.sort()
            self.update_x()
            
    def calc_fitness(self, A, b, eps):
        if not is_feasible(self.x, A, b, eps):
            return float('inf')
        return len(self.cols)

In [4]:
def selection(population, tournament_size, A, b, eps):
    chosen = random.sample(population, tournament_size)
    return min(chosen, key= lambda x: x.calc_fitness(A, b, eps))

In [5]:
def crossover(parent1, parent2, child, A, b):
    child.cols = list(set(parent1.cols) | set(parent2.cols)) #unija kolona
    child.update_x(A, b)

In [6]:
def mutation(individual, mutation_prob, A, b):
    n = A.shape[1]
    for i in range(len(individual.cols)):
        if random.random() < mutation_prob:
            individual.cols[i] = random.randint(0, n-1)
    individual.cols = list(set(individual.cols))
    individual.update_x(A, b)

In [7]:

def ga(A, b, eps, population_size, num_generations, tournament_size, elitism_size, mutation_prob):
    population = [Individual(A, b, eps) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size):
            parent1 = selection(population, tournament_size, A, b, eps)
            parent2 = parent1
            while parent2 == parent1:
                parent2 = selection(population, tournament_size, A, b, eps) 
            
            crossover(parent1, parent2, new_population[j], A, b)

            mutation(new_population[j], mutation_prob, A, b)
            
            new_population[j].fitness = new_population[j].calc_fitness(A, b, eps)
        
        population = new_population.copy()
    return min(population, key=lambda x: x.fitness)

In [ ]:
results_small = []
times_small = []
for i in range(5):
    A = np.load(f'../data/small_data/{i}_A.npy')  
    b = np.load(f'../data/small_data/{i}_b.npy')  

    _, res, _, _ = np.linalg.lstsq(A, b, rcond=None)
    eps = res * 1.01

    start_time = time.time()
    best_individual = ga(A, b, eps,population_size=100,
    num_generations=200,
    tournament_size=10,
    elitism_size=10,
    mutation_prob=0.002,)
    end_time = time.time()
    results_small.append(best_individual.fitness)
    times_small.append(end_time-start_time)



In [ ]:
np.save('../tests/ga_results_small', results_small)
np.save('../tests/ga_times_small', times_small)

In [ ]:
results_small

In [ ]:
times_small

In [ ]:
results_big = []
times_big = []
for i in range(5):
    A = np.load(f'../data/big_data/{i}_A.npy')  
    b = np.load(f'../data/big_data/{i}_b.npy')  

    _, res, _, _ = np.linalg.lstsq(A, b, rcond=None)
    eps = res * 1.01

    start_time = time.time()
    best_individual = ga(A, b, eps,population_size=100,
    num_generations=200,
    tournament_size=7,
    elitism_size=10,
    mutation_prob=0.002,)
    end_time = time.time()
    results_big.append(best_individual.fitness)
    times_big.append(end_time-start_time)



In [ ]:
np.save('../tests/ga_times_big', times_big)
np.save('../tests/ga_results_big', results_big)

In [ ]:
results_big

In [ ]:
times_big